# Xuất mô hình từ PyTorch sang ONNX và chạy với ONNX Runtime
Hướng dẫn này sẽ hướng dẫn cách xuất một mô hình Super Resolution từ PyTorch sang ONNX và kiểm tra với ONNX Runtime.

## 1. Cài đặt thư viện cần thiết
Chúng ta cần cài đặt PyTorch, ONNX và ONNX Runtime.

In [1]:
!pip install torch torchvision onnx onnxruntime pillow numpy

## 2. Import các thư viện cần thiết

In [2]:

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.utils.model_zoo as model_zoo
import onnx
import onnxruntime
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
import time
    

## 3. Định nghĩa mô hình Super Resolution trong PyTorch

In [3]:

class SuperResolutionNet(nn.Module):
    def __init__(self, upscale_factor, inplace=False):
        super(SuperResolutionNet, self).__init__()

        self.relu = nn.ReLU(inplace=inplace)
        self.conv1 = nn.Conv2d(1, 64, (5, 5), (1, 1), (2, 2))
        self.conv2 = nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1))
        self.conv3 = nn.Conv2d(64, 32, (3, 3), (1, 1), (1, 1))
        self.conv4 = nn.Conv2d(32, upscale_factor ** 2, (3, 3), (1, 1), (1, 1))
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)

        self._initialize_weights()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.pixel_shuffle(self.conv4(x))
        return x

    def _initialize_weights(self):
        init.orthogonal_(self.conv1.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv2.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv3.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv4.weight)
    

## 4. Tạo mô hình và tải trọng số đã huấn luyện trước

In [4]:

model_url = 'https://s3.amazonaws.com/pytorch/test_data/export/superres_epoch100-44c6958e.pth'
torch_model = SuperResolutionNet(upscale_factor=3)
map_location = lambda storage, loc: storage
torch_model.load_state_dict(model_zoo.load_url(model_url, map_location=map_location))
torch_model.eval()
    

SuperResolutionNet(
  (relu): ReLU()
  (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(32, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pixel_shuffle): PixelShuffle(upscale_factor=3)
)

## 5. Xuất mô hình sang định dạng ONNX

In [5]:

x = torch.randn(1, 1, 224, 224, requires_grad=True)
torch.onnx.export(torch_model, x, "super_resolution.onnx", export_params=True, opset_version=10,
                  do_constant_folding=True, input_names=['input'], output_names=['output'],
                  dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})
    

## 6. Kiểm tra mô hình ONNX

In [6]:

onnx_model = onnx.load("super_resolution.onnx")
onnx.checker.check_model(onnx_model)
    

## 7. Chạy mô hình với ONNX Runtime

In [7]:

ort_session = onnxruntime.InferenceSession("super_resolution.onnx", providers=["CPUExecutionProvider"])

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

np.testing.assert_allclose(to_numpy(torch_model(x)), ort_outs[0], rtol=1e-03, atol=1e-05)
print("Mô hình ONNX chạy tốt với ONNX Runtime!")
    

Mô hình ONNX chạy tốt với ONNX Runtime!


## 8. So sánh hiệu suất giữa PyTorch và ONNX Runtime

In [8]:

x = torch.randn(1, 1, 224, 224, requires_grad=True)

start = time.time()
torch_out = torch_model(x)
end = time.time()
print(f"Inference với PyTorch mất {end - start} giây")

ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
start = time.time()
ort_outs = ort_session.run(None, ort_inputs)
end = time.time()
print(f"Inference với ONNX Runtime mất {end - start} giây")
    

Inference với PyTorch mất 0.03198099136352539 giây
Inference với ONNX Runtime mất 0.02173304557800293 giây


# 9. Mở ảnh và tiền xử lý

In [9]:
img = Image.open("cat.jpg")

resize = transforms.Resize([224, 224])
img = resize(img)

img_ycbcr = img.convert('YCbCr')
img_y, img_cb, img_cr = img_ycbcr.split()

to_tensor = transforms.ToTensor()
img_y = to_tensor(img_y)
img_y.unsqueeze_(0)

tensor([[[[0.8039, 0.8039, 0.8039,  ..., 0.7961, 0.7961, 0.7961],
          [0.8118, 0.8078, 0.8118,  ..., 0.7922, 0.7922, 0.7922],
          [0.8078, 0.8078, 0.8039,  ..., 0.7922, 0.7922, 0.7922],
          ...,
          [0.6275, 0.6353, 0.6353,  ..., 0.6431, 0.6431, 0.6353],
          [0.6353, 0.6353, 0.6353,  ..., 0.6471, 0.6431, 0.6431],
          [0.6471, 0.6392, 0.6353,  ..., 0.6471, 0.6471, 0.6431]]]])

# 10. Chạy mô hình ONNX trên ảnh

In [10]:
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(img_y)}
ort_outs = ort_session.run(None, ort_inputs)
img_out_y = ort_outs[0] 

# 11. Hậu xử lý và lưu ảnh kết quả

In [11]:
img_out_y = Image.fromarray(np.uint8((img_out_y[0] * 255.0).clip(0, 255)[0]), mode='L')
final_img = Image.merge(
    "YCbCr", [
        img_out_y,
        img_cb.resize(img_out_y.size, Image.BICUBIC),
        img_cr.resize(img_out_y.size, Image.BICUBIC),
    ]).convert("RGB")

final_img.save("cat_superres_with_ort.jpg")
img = transforms.Resize([img_out_y.size[0], img_out_y.size[1]])(img)
img.save("cat_resized.jpg")
